In [25]:
import os

import pandas as pd

import yfinance as yf
from datetime import datetime, timedelta

from tqdm import tqdm

In [19]:
# Load ticker symbols from nasdaq screener csv file
def load_tickers(filepath):
    df = pd.read_csv(filepath)
    return df.iloc[:, 0].tolist()

In [20]:
# Fetch historical data for a given ticker
def fetch_stock_data(ticker):
    # Calculate the date 10 years ago from today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*10)
    
    # Fetch historical data
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return data

In [26]:
# Load ticker symbols
tickers = load_tickers('nasdaq_screener.csv')

# Create a subfolder for the financial data
output_dir = 'fin_data'
os.makedirs(output_dir, exist_ok=True)

# Dictionary to hold data for all tickers
all_data = {}

# Loop through each ticker and fetch its data
for ticker in tqdm(tickers):
    #print(f"Fetching data for: {ticker}")
    try:
        data = fetch_stock_data(ticker)
        all_data[ticker] = data
        # Optionally save data to CSV (comment out if not needed)
        data.to_csv(os.path.join(output_dir, f'{ticker}_10_years_data.csv'))
    except Exception as e:
        print(f"Failed to fetch data for {ticker}: {e}")

# Example of how to access data for a specific ticker in memory
# print(all_data['AAPL'])  # Uncomment to print AAPL data

  0%|          | 0/7070 [00:00<?, ?it/s]

Fetching data for: A
[*********************100%***********************]  1 of 1 completed

  0%|          | 1/7070 [00:00<18:37,  6.33it/s]


Failed to fetch data for A: 'A'
Fetching data for: AA
[*********************100%***********************]  1 of 1 completed
Failed to fetch data for AA: 'AA'
Fetching data for: AACG
[*********************100%***********************]  1 of 1 completed

  0%|          | 3/7070 [00:00<09:40, 12.18it/s]

...


Fetching data for: ADIL
[*********************100%***********************]  1 of 1 completed

  1%|▏         | 105/7070 [00:08<15:16,  7.60it/s]


Fetching data for: ADM


  1%|▏         | 105/7070 [00:08<09:38, 12.03it/s]


KeyboardInterrupt: 